<a href="https://colab.research.google.com/github/benebobaa/ML-Movie_Recommendation/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Movie Recomendation Engine with Collaborative Filltering Using Spark.**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf /content/drive/MyDrive/spark-3.0.0-bin-hadoop3.2.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=93675ff2effe9225cc8bb2722e80d8c9100ef08a33b7837c6d16589d43f8e6d8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


#Import Library

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator            #Alternating Least Square
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.recommendation import ALS                        # #Alternating Least Square
from pyspark.sql.functions import udf, col, when, count, isnan, row_number, window, explode, rand
import random

#Import Dataset from Kaggle

In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"benediktussatriya","key":"5571ef5062bae925b043b79195cc175c"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

 96% 188M/195M [00:02<00:00, 104MB/s] 
100% 195M/195M [00:02<00:00, 93.2MB/s]


In [ ]:
!unzip /content/movielens-20m-dataset.zip

Archive:  /content/movielens-20m-dataset.zip
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              
  inflating: tag.csv                 


In [ ]:
spark = SparkSession.builder.appName("movie_recomendation").getOrCreate()
print(spark.version)

3.3.1


###Read Dataset and Joining the datasets

In [ ]:
df_rating = spark.read.csv("/content/rating.csv",inferSchema=True, header=True)
df_movie = spark.read.csv("/content/movie.csv",inferSchema=True, header=True)

df_movie_ratings = df_rating.join(df_movie, ['movieId'], 'left')
df_movie_ratings.show(10)

+-------+------+------+-------------------+--------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|              genres|
+-------+------+------+-------------------+--------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|Adventure|Childre...|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|Adventure|Drama|F...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|Crime|Mystery|Thr...|
|    112|     1|   3.5|2004-09-10 03:09:00|Rumble in the Bro...|Action|Adventure|...|
|    151|     1|   4.0|2004-09-10 03:08:54|      Rob Roy (1995)|Action|Drama|Roma...|
|    223|     1|   4.0|2005-04-02 23:46:13|       Clerks (1994)|              Comedy|
|    253|     1|   4.0|2005-04-02 23:35:40|Interview w

### Choose quer from dataset where userId around 1-10, because the origin datasets heavier

In [ ]:
#Create newest data fr
df_movie_ratings.createOrReplaceTempView("df_view")
df_movie_new = spark.sql(
    """
    SELECT * 
    FROM df_view
    WHERE userId <= 10
    """
)

In [ ]:
df_movie_new.show(10)

+-------+------+------+-------------------+--------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|              genres|
+-------+------+------+-------------------+--------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|Adventure|Childre...|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|Adventure|Drama|F...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|Crime|Mystery|Thr...|
|    112|     1|   3.5|2004-09-10 03:09:00|Rumble in the Bro...|Action|Adventure|...|
|    151|     1|   4.0|2004-09-10 03:08:54|      Rob Roy (1995)|Action|Drama|Roma...|
|    223|     1|   4.0|2005-04-02 23:46:13|       Clerks (1994)|              Comedy|
|    253|     1|   4.0|2005-04-02 23:35:40|Interview w

In [ ]:
df_movie_new.describe().show()

+-------+------------------+------------------+------------------+--------------------+----------------+
|summary|           movieId|            userId|            rating|               title|          genres|
+-------+------------------+------------------+------------------+--------------------+----------------+
|  count|               960|               960|               960|                 960|             960|
|   mean|2048.0510416666666| 4.823958333333334|3.7197916666666666|                null|            null|
| stddev| 1927.862911118332|2.7709292603631606|0.9142733077505908|                null|            null|
|    min|                 1|                 1|               1.0|1984 (Nineteen Ei...|Action|Adventure|
|    max|             31696|                10|               5.0|¡Three Amigos! (1...|         Western|
+-------+------------------+------------------+------------------+--------------------+----------------+



In [ ]:
print("Shape of the dataset: ", (df_movie_new.count(), len(df_movie_new.columns)))

Shape of the dataset:  (960, 6)


In [ ]:
df_movie_new.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



Remove column timestamp because we not used that

In [ ]:
df_movie_new = df_movie_new.drop("timestamp") 
df_movie_new.show(10)

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|      2|     1|   3.5|      Jumanji (1995)|Adventure|Childre...|
|     29|     1|   3.5|City of Lost Chil...|Adventure|Drama|F...|
|     32|     1|   3.5|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|     1|   3.5|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   3.5|Usual Suspects, T...|Crime|Mystery|Thr...|
|    112|     1|   3.5|Rumble in the Bro...|Action|Adventure|...|
|    151|     1|   4.0|      Rob Roy (1995)|Action|Drama|Roma...|
|    223|     1|   4.0|       Clerks (1994)|              Comedy|
|    253|     1|   4.0|Interview with th...|        Drama|Horror|
|    260|     1|   4.0|Star Wars: Episod...|Action|Adventure|...|
+-------+------+------+--------------------+--------------------+
only showing top 10 rows



check null value

In [ ]:
df_new_null = df_movie_new.select([count(when(col(c).contains("None")|col(c).contains("NULL")|(col(c) =="")|col(c).isNull()|isnan(c),c)).alias(c)for c in df_movie_new.columns])
df_new_null.show(10)

+-------+------+------+-----+------+
|movieId|userId|rating|title|genres|
+-------+------+------+-----+------+
|      0|     0|     0|    0|     0|
+-------+------+------+-----+------+



#Spliting Data

In [ ]:
# Create test and train set
training_df, test_df = df_movie_new.randomSplit([0.8, 0.2], seed = 2020)

Check training_df not empty

In [ ]:
training_df.show(5)

+-------+------+------+----------------+--------------------+
|movieId|userId|rating|           title|              genres|
+-------+------+------+----------------+--------------------+
|      1|     3|   4.0|Toy Story (1995)|Adventure|Animati...|
|      1|     6|   5.0|Toy Story (1995)|Adventure|Animati...|
|      1|     8|   4.0|Toy Story (1995)|Adventure|Animati...|
|      1|    10|   4.0|Toy Story (1995)|Adventure|Animati...|
|      2|     1|   3.5|  Jumanji (1995)|Adventure|Childre...|
+-------+------+------+----------------+--------------------+
only showing top 5 rows




Check training_df not empty

In [ ]:
test_df.show(5)

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|      3|     7|   3.0|Grumpier Old Men ...|      Comedy|Romance|
|      7|     6|   5.0|      Sabrina (1995)|      Comedy|Romance|
|     11|     5|   5.0|American Presiden...|Comedy|Drama|Romance|
|     15|     7|   2.0|Cutthroat Island ...|Action|Adventure|...|
|     16|     7|   3.0|       Casino (1995)|         Crime|Drama|
+-------+------+------+--------------------+--------------------+
only showing top 5 rows



#Modeling with ALS

Put the column for make predict 

In [ ]:
# Create ALS model
als = ALS(
         userCol="userId", 
         itemCol="movieId",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)


In [ ]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [ ]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 

In [ ]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

Training data

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(training_df)

#Extract best model from the cv model above
best_model = model.bestModel

# View the predictions
test_predictions = best_model.transform(test_df)

Evaluation with RMSE

In [ ]:
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

1.1951181875332397


#Test the recommendation system

In [ ]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(5)
nrecommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    10|[{1221, 4.824484}...|
|     1|[{8507, 4.8168077...|
|     2|[{457, 4.945968},...|
|     3|[{858, 4.951282},...|
|     4|[{454, 4.608115},...|
|     5|[{1196, 4.9781013...|
|     6|[{52, 4.8815203},...|
|     7|[{1077, 4.821388}...|
|     8|[{527, 4.8993764}...|
|     9|[{4148, 4.8266053...|
+------+--------------------+



In [ ]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|    10|   1221| 4.824484|
|    10|   1247| 4.824484|
|    10|    858|4.7805624|
|    10|    527|4.7575636|
|    10|   1387|4.6533937|
|     1|   8507|4.8168077|
|     1|   4993|4.8168077|
|     1|   5952|4.8168077|
|     1|   7153|4.8168077|
|     1|    858|4.3559747|
+------+-------+---------+



In [ ]:
nrecommendations.join(df_movie, on='movieId').filter('userId = 1').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   8507|     1|4.8168077|       Freaks (1932)|  Crime|Drama|Horror|
|   4993|     1|4.8168077|Lord of the Rings...|   Adventure|Fantasy|
|   5952|     1|4.8168077|Lord of the Rings...|   Adventure|Fantasy|
|   7153|     1|4.8168077|Lord of the Rings...|Action|Adventure|...|
|    858|     1|4.3559747|Godfather, The (1...|         Crime|Drama|
+-------+------+---------+--------------------+--------------------+



In [ ]:
df_movie_new.join(df_movie, on='movieId').filter('userId = 10').sort('rating', ascending=False).limit(5).show()

+-------+------+------+--------------------+--------------------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|               title|              genres|
+-------+------+------+--------------------+--------------------+--------------------+--------------------+
|    527|    10|   5.0|Schindler's List ...|           Drama|War|Schindler's List ...|           Drama|War|
|    858|    10|   5.0|Godfather, The (1...|         Crime|Drama|Godfather, The (1...|         Crime|Drama|
|   1221|    10|   5.0|Godfather: Part I...|         Crime|Drama|Godfather: Part I...|         Crime|Drama|
|   1247|    10|   5.0|Graduate, The (1967)|Comedy|Drama|Romance|Graduate, The (1967)|Comedy|Drama|Romance|
|   1387|    10|   5.0|         Jaws (1975)|       Action|Horror|         Jaws (1975)|       Action|Horror|
+-------+------+------+--------------------+--------------------+--------------------+--------------------+



Recommendation system by random user id range 1 to 10

In [ ]:
user = random.randint(1, 10)
nrecommendations.join(df_movie, on='movieId').filter('userId = {}'.format(user)).show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   1221|    10| 4.824484|Godfather: Part I...|         Crime|Drama|
|   1247|    10| 4.824484|Graduate, The (1967)|Comedy|Drama|Romance|
|    858|    10|4.7805624|Godfather, The (1...|         Crime|Drama|
|    527|    10|4.7575636|Schindler's List ...|           Drama|War|
|   1387|    10|4.6533937|         Jaws (1975)|       Action|Horror|
+-------+------+---------+--------------------+--------------------+



Recommendation system by best rating prediction random rows

In [ ]:
nrecommendations.join(df_movie, on='movieId').orderBy(rand()).limit(5).show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|    434|     8| 4.878001|  Cliffhanger (1993)|Action|Adventure|...|
|   4148|     9|4.8266053|     Hannibal (2001)|     Horror|Thriller|
|   1256|     7| 4.821388|    Duck Soup (1933)|  Comedy|Musical|War|
|   7153|     1|4.8168077|Lord of the Rings...|Action|Adventure|...|
|   1077|     7| 4.821388|      Sleeper (1973)|       Comedy|Sci-Fi|
+-------+------+---------+--------------------+--------------------+



#Conclusion
The value rating between result predict and actual data from dataset is very close. That mean the prediction is good.